# Classification de documents PDF techniques : PC vs Moniteurs

## 1. Introduction

- Objectif : Déterminer automatiquement si un document PDF technique concerne un PC ou un moniteur.

- Intérêt : Automatiser un tri documentaire pour gagner du temps dans l’archivage ou la recherche de fichiers.

## 2. Librairies

In [17]:
#!pip install -r requirements.txt

## 3. Création de la database

Un total de 542 fichiers PDF ont été collectés. Il s'agit de fiches PCF (Product Carbon Footprint) portant sur des ordinateurs ou des écrans (moniteurs) de la marque HP. Ces documents ont été récupérés directement depuis le site officiel de HP à l’aide d’une méthode de web scraping, dont le code est présenté ci-dessous. Le processus s’effectue en deux étapes : d’abord, les liens vers les fichiers PDF sont extraits et stockés dans un objet ; ensuite, les fichiers sont téléchargés localement.

### 3.1 Téléchargement des écrans

monitor : https://h20195.www2.hp.com/v2/library.aspx?doctype=95&footer=95&filter_doctype=no&filter_country=no&cc=us&lc=en&filter_oid=no&filter_prodtype=rw&prodtype=ij&showproductcompatibility=yes&showregion=yes&showreglangcol=yes&showdescription=yes3doctype-95&sortorder-popular&teasers-off&isRetired-false&isRHParentNode-false&titleCheck-false#doctype-95&product_type-mo&sortorder-document_title&teasers-off&isRetired-false&isRHParentNode-false&titleCheck-false

In [4]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Initialiser le navigateur
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

hp_root_url = "https://h20195.www2.hp.com/v2/library.aspx?doctype=95&footer=95&filter_doctype=no&filter_country=no&cc=us&lc=en&filter_oid=no&filter_prodtype=rw&prodtype=ij&showproductcompatibility=yes&showregion=yes&showreglangcol=yes&showdescription=yes3doctype-95&sortorder-popular&teasers-off&isRetired-false&isRHParentNode-false&titleCheck-false#doctype-95&product_type-mo&sortorder-document_title&teasers-off&isRetired-false&isRHParentNode-false&titleCheck-false"
driver.get(hp_root_url)

# Accepter les cookies
try:
    cookie_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Accept")]')))
    cookie_button.click()
    print("Cookies acceptés.")
except:
    print("Pas de cookies à accepter.")

# Cliquer sur "Load More" tant qu'il existe
while True:
    try:
        load_more_btn = wait.until(EC.element_to_be_clickable((By.ID, "loadMore")))
        driver.execute_script("arguments[0].click();", load_more_btn)
        print("→ Clicked 'Load More'")
        time.sleep(2)  # petite pause pour chargement
    except:
        print("✓ Plus de bouton 'Load More' détecté.")
        break

# Attendre que tout soit chargé
time.sleep(5)

# Extraire le HTML final
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Récupérer tous les liens PDF via l'attribut "pdfurl"
pdf_links = []
for link in soup.find_all('a'):
    pdfurl = link.get('pdfurl')
    if pdfurl:
        full_url = urljoin(hp_root_url, pdfurl)
        pdf_links.append(full_url)

# Afficher les liens
print(f"{len(pdf_links)} PDF trouvés :")
for url in pdf_links[:5]:
    print(url)

# Fermer le navigateur
driver.quit()

Cookies acceptés.
→ Clicked 'Load More'
→ Clicked 'Load More'
✓ Plus de bouton 'Load More' détecté.
280 PDF trouvés :
https://h20195.www2.hp.com/v2/GetDocument.aspx?docname=c08699464
https://h20195.www2.hp.com/v2/GetDocument.aspx?docname=c08361997
https://h20195.www2.hp.com/v2/GetDocument.aspx?docname=c09125938
https://h20195.www2.hp.com/v2/GetDocument.aspx?docname=c08881237
https://h20195.www2.hp.com/v2/GetDocument.aspx?docname=c08881231


In [ ]:
import os
import requests

# Chemin local Windows vers ton dossier
save_dir = r"C:\Users\simdu\OneDrive\Documents\M2\S2\NLP_HAMZA\project\data\monitors"

# Créer le dossier s'il n'existe pas
os.makedirs(save_dir, exist_ok=True)

# Télécharger tous les PDF
for i, url in enumerate(pdf_links, start=1):
    try:
        response = requests.get(url)
        response.raise_for_status()
        # Essayer d'extraire un nom de fichier depuis l'URL
        filename = url.split('/')[-1]
        if not filename.endswith(".pdf"):
            filename = f"hp_doc_{i}.pdf"
        filepath = os.path.join(save_dir, filename)
        with open(filepath, 'wb') as f:
            f.write(response.content)
        print(f"✅ Téléchargé : {filepath}")
    except Exception as e:
        print(f"❌ Échec pour {url} : {e}")


### 3.1 Téléchargement des PC

pc : https://h20195.www2.hp.com/v2/library.aspx?doctype=95&footer=95&filter_doctype=no&filter_country=no&cc=us&lc=en&filter_oid=no&filter_prodtype=rw&prodtype=ij&showproductcompatibility=yes&showregion=yes&showreglangcol=yes&showdescription=yes3doctype-95&sortorder-popular&teasers-off&isRetired-false&isRHParentNode-false&titleCheck-false#doctype-95&product_type-pc&sortorder-document_title&teasers-off&isRetired-false&isRHParentNode-false&titleCheck-false

In [7]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Initialiser le navigateur
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

hp_pc_root_url = "https://h20195.www2.hp.com/v2/library.aspx?doctype=95&footer=95&filter_doctype=no&filter_country=no&cc=us&lc=en&filter_oid=no&filter_prodtype=rw&prodtype=ij&showproductcompatibility=yes&showregion=yes&showreglangcol=yes&showdescription=yes3doctype-95&sortorder-popular&teasers-off&isRetired-false&isRHParentNode-false&titleCheck-false#doctype-95&product_type-pc&sortorder-document_title&teasers-off&isRetired-false&isRHParentNode-false&titleCheck-false"
driver.get(hp_pc_root_url)

# Accepter les cookies
try:
    cookie_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Accept")]')))
    cookie_button.click()
    print("Cookies acceptés.")
except:
    print("Pas de cookies à accepter.")

# Cliquer sur "Load More" tant qu'il existe
while True:
    try:
        load_more_btn = wait.until(EC.element_to_be_clickable((By.ID, "loadMore")))
        driver.execute_script("arguments[0].click();", load_more_btn)
        print("→ Clicked 'Load More'")
        time.sleep(2)  # petite pause pour chargement
    except:
        print("✓ Plus de bouton 'Load More' détecté.")
        break

# Attendre que tout soit chargé
time.sleep(5)

# Extraire le HTML final
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Récupérer tous les liens PDF via l'attribut "pdfurl"
pdf_links = []
for link in soup.find_all('a'):
    pdfurl = link.get('pdfurl')
    if pdfurl:
        full_url = urljoin(hp_root_url, pdfurl)
        pdf_links.append(full_url)

# Afficher les liens
print(f"{len(pdf_links)} PDF trouvés :")
for url in pdf_links[:5]:
    print(url)

# Fermer le navigateur
driver.quit()

Cookies acceptés.
→ Clicked 'Load More'
→ Clicked 'Load More'
✓ Plus de bouton 'Load More' détecté.
262 PDF trouvés :
https://h20195.www2.hp.com/v2/GetDocument.aspx?docname=c07524666
https://h20195.www2.hp.com/v2/GetDocument.aspx?docname=c07524664
https://h20195.www2.hp.com/v2/GetDocument.aspx?docname=c07524680
https://h20195.www2.hp.com/v2/GetDocument.aspx?docname=c07524714
https://h20195.www2.hp.com/v2/GetDocument.aspx?docname=c07524681


In [ ]:
import os
import requests

# Chemin local Windows vers ton dossier
save_dir = r"C:\Users\simdu\OneDrive\Documents\M2\S2\NLP_HAMZA\project\data\pc"

# Créer le dossier s'il n'existe pas
os.makedirs(save_dir, exist_ok=True)

# Numéro de départ pour les fichiers
start_index = 281

# Télécharger tous les PDF
for i, url in enumerate(pdf_links, start=start_index):
    try:
        response = requests.get(url)
        response.raise_for_status()
        filename = f"hp_doc_{i}.pdf"
        filepath = os.path.join(save_dir, filename)
        with open(filepath, 'wb') as f:
            f.write(response.content)
        print(f"✅ Téléchargé : {filepath}")
    except Exception as e:
        print(f"❌ Échec pour {url} : {e}")

### 3.3 Extraction du texte et création de la database

In [18]:
import os
import PyPDF2
import pandas as pd
from tqdm import tqdm

def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text()  # Extraire le texte de chaque page
        return text
    except Exception as e:
        print(f"Erreur avec {pdf_path} : {e}")
        return ""

# Répertoires
base_dir = r"C:\Users\simdu\OneDrive\Documents\M2\S2\NLP_HAMZA\project\data"
folders = ["pc", "monitors"]

data = []

for label in folders:
    folder_path = os.path.join(base_dir, label)
    for filename in tqdm(os.listdir(folder_path), desc=f"Traitement des fichiers {label}"):
        if filename.endswith(".pdf"):
            file_path = os.path.join(folder_path, filename)
            text = extract_text_from_pdf(file_path)
            data.append({
                "filename": filename,
                "text": text,
                "label": label
            })

# Convertir en DataFrame
df = pd.DataFrame(data)

# Sauvegarder en CSV pour usage futur
df.to_csv("pdf_texts.csv", index=False)
print("✅ Extraction terminée et CSV enregistré.")



Traitement des fichiers monitors:  33%|███▎      | 93/280 [00:37<09:36,  3.08s/it]

Erreur avec C:\Users\simdu\OneDrive\Documents\M2\S2\NLP_HAMZA\project\data\monitors\hp_doc_183.pdf : Cannot read an empty file


Traitement des fichiers monitors: 100%|██████████| 280/280 [01:44<00:00,  2.69it/s]

✅ Extraction terminée et CSV enregistré.


### 3.4 Traitement de la database

In [32]:
import pandas as pd

df = pd.read_csv("pdf_texts.csv")
print(df.head())

         filename                                               text label
0  hp_doc_281.pdf  Product Carbon Footprint Report\n23-Aug-2023\n...    pc
1  hp_doc_282.pdf   \n Product carbon footprint  \nHP 200 Pro G4 ...    pc
2  hp_doc_283.pdf   \n Product carbon footprint  \nHP 20 5 G4 22 ...    pc
3  hp_doc_284.pdf   \n Product carbon footprint  \nHP 205 G4 24 A...    pc
4  hp_doc_285.pdf   \n Product carbon footprint  \nHP 20 5 Pro G4...    pc


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542 entries, 0 to 541
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  542 non-null    object
 1   text      530 non-null    object
 2   label     542 non-null    object
dtypes: object(3)
memory usage: 12.8+ KB


On voit qu'il y a 542-530 = 12 fichiers dont le texte n'a pas pu être extrait. Choix de suppression.

In [35]:
# Supprimer directement les fichiers sans texte extrait dans df
df_clean = df.dropna(subset=['text'])

# Vérifier la nouvelle taille du DataFrame
print(df_clean.shape)

# Sauvegarder le DataFrame nettoyé en CSV
df_clean.to_csv("pdf_texts_cleaned.csv", index=False)
print("✅ Fichiers sans texte supprimés et CSV nettoyé sauvegardé.")



(530, 3)
✅ Fichiers sans texte supprimés et CSV nettoyé sauvegardé.


In [36]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 530 entries, 0 to 541
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  530 non-null    object
 1   text      530 non-null    object
 2   label     530 non-null    object
dtypes: object(3)
memory usage: 16.6+ KB


In [37]:
df_clean['label'].value_counts()

label
monitors    269
pc          261
Name: count, dtype: int64